In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar100
from sklearn.preprocessing import LabelBinarizer

In [6]:
# Load CIFAR-100 Dataset
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

# Normalize pixel values between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-Hot Encode Labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape: {X_test.shape}")


Training Data Shape: (50000, 32, 32, 3)
Test Data Shape: (10000, 32, 32, 3)
